# Predict Mask

<span style="font-size: 20px; line-height: 30px;">
<ol>
    <li> Load model </li>
    <li> Load test dataset - Images </li>
    <li> Predict the mask </li>
    <li> Calculate FPS </li>
</ol>
</span>

## Import

In [1]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope

## Seeding

In [2]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

## Hyperparameters

In [3]:
height = 384
width = 512

## Path

In [4]:
dataset_path = "dataset"
save_path = "prediction"

model_path = os.path.join("files", "unet.h5")

## Create folder

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

## Folder to save predicted mask

In [6]:
create_dir(save_path)

## Load model

In [7]:
model = tf.keras.models.load_model(model_path)

2022-10-09 11:38:27.840908: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 11:38:27.871676: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 11:38:27.872184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 11:38:27.873111: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 390, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 192, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                               

                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 96, 128, 64)  16448       ['conv2_block2_out[0][0]']       
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 96, 128, 64)  256        ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 96, 128, 64)  0          ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 96, 128, 64)  36928       ['conv2_block3_1_relu[0][0]']    
          

                                                                                                  
 conv3_block2_out (Activation)  (None, 48, 64, 512)  0           ['conv3_block2_add[0][0]']       
                                                                                                  
 conv3_block3_1_conv (Conv2D)   (None, 48, 64, 128)  65664       ['conv3_block2_out[0][0]']       
                                                                                                  
 conv3_block3_1_bn (BatchNormal  (None, 48, 64, 128)  512        ['conv3_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 48, 64, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
          

 conv4_block1_add (Add)         (None, 24, 32, 1024  0           ['conv4_block1_0_bn[0][0]',      
                                )                                 'conv4_block1_3_bn[0][0]']      
                                                                                                  
 conv4_block1_out (Activation)  (None, 24, 32, 1024  0           ['conv4_block1_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block2_1_conv (Conv2D)   (None, 24, 32, 256)  262400      ['conv4_block1_out[0][0]']       
                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 24, 32, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block4_add (Add)         (None, 24, 32, 1024  0           ['conv4_block3_out[0][0]',       
                                )                                 'conv4_block4_3_bn[0][0]']      
                                                                                                  
 conv4_block4_out (Activation)  (None, 24, 32, 1024  0           ['conv4_block4_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block5_1_conv (Conv2D)   (None, 24, 32, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 24, 32, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization) 

 global_average_pooling2d (Glob  (None, 512)         0           ['activation_1[0][0]']           
 alAveragePooling2D)                                                                              
                                                                                                  
 global_max_pooling2d (GlobalMa  (None, 512)         0           ['activation_1[0][0]']           
 xPooling2D)                                                                                      
                                                                                                  
 dense (Dense)                  (None, 64)           32768       ['global_average_pooling2d[0][0]'
                                                                 , 'global_max_pooling2d[0][0]']  
                                                                                                  
 dense_1 (Dense)                (None, 512)          32768       ['dense[0][0]',                  
          

 tf.math.reduce_mean_1 (TFOpLam  (None, 96, 128)     0           ['multiply_2[0][0]']             
 bda)                                                                                             
                                                                                                  
 tf.math.reduce_max_1 (TFOpLamb  (None, 96, 128)     0           ['multiply_2[0][0]']             
 da)                                                                                              
                                                                                                  
 tf.expand_dims_2 (TFOpLambda)  (None, 96, 128, 1)   0           ['tf.math.reduce_mean_1[0][0]']  
                                                                                                  
 tf.expand_dims_3 (TFOpLambda)  (None, 96, 128, 1)   0           ['tf.math.reduce_max_1[0][0]']   
                                                                                                  
 concatena

                                                                                                  
 concatenate_6 (Concatenate)    (None, 384, 512, 67  0           ['conv2d_transpose_3[0][0]',     
                                )                                 'input_1[0][0]']                
                                                                                                  
 conv2d_9 (Conv2D)              (None, 384, 512, 64  38656       ['concatenate_6[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 384, 512, 64  256        ['conv2d_9[0][0]']               
 rmalization)                   )                                                                 
                                                                                                  
 activatio

## Load test dataset: Images

In [9]:
test_x = sorted(glob(os.path.join(dataset_path, "test", "images", "*")))
print("Test Images: ", len(test_x))

Test Images:  70


## Predict the Mask

In [10]:
time_taken = []

for x in tqdm(test_x):
    name = x.split("/")[-1]
    
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    h, w, _ = x.shape
    x = cv2.resize(x, (width, height))
    x = x/255.0
    x = np.expand_dims(x, axis=0)
    
    start_time = time.time()
    p = model.predict(x, verbose=0)[0]
    total_time = time.time() - start_time
    
    time_taken.append(total_time)
    
    p = cv2.resize(p, (w, h))
    p = np.argmax(p, axis=-1)
    p = p.astype(np.float32)
    
    cv2.imwrite(os.path.join(save_path, name), p)

  0%|                                                                                                                                                                               | 0/70 [00:00<?, ?it/s]2022-10-09 11:41:36.382787: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-10-09 11:41:38.031215: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [00:07<00:00,  9.91it/s]


In [11]:
mean_time = np.mean(time_taken)
mean_fps = 1/mean_time

print(f"Mean Time: {mean_time: 1.5f} - Mean FPS: {mean_fps:2.5f}")

Mean Time:  0.08214 - Mean FPS: 12.17465
